<a href="https://colab.research.google.com/github/peculab/AI4JUBO/blob/main/JuboDeath_V7_SMD_pureData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install shap plotly xgboost --quiet

In [2]:
!pip uninstall shap -y
!pip install shap --no-deps

Found existing installation: shap 0.48.0
Uninstalling shap-0.48.0:
  Successfully uninstalled shap-0.48.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.1 MB/s eta 0:00:00


In [3]:
!pip install ace_tools

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from IPython.display import display
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    classification_report, confusion_matrix, mean_absolute_error, r2_score
)

In [5]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [6]:
# read data and put it in a dataframe
# 在 google 工作表載入外部資料 gsheets

gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1W93D0WRk2GYeWnT8fOsJLwbYlbzJNXJJco1pYUrlZrU/edit?usp=sharing')
worksheet = gsheets.worksheet("external_validation_0820")  # 指定分頁名稱

worksheet = worksheet.get_all_records()
external = pd.DataFrame(worksheet)
external = external.apply(lambda col: pd.to_numeric(col.astype(str).str.replace(',', '').str.strip(), errors='coerce'))
external.head()

,H01_NUM,dbname,入家日期,結案日期,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,ADL_first_score,ADL_last_score,ADL_diff,ADL_std,ADL_明顯惡化,意識清醒_max,使用呼吸輔具,活動假牙,使用管路,六個月內住院次數
0,1326,NaN,NaN,NaN,1,32,0,93,0,0,0,0,0,NaN,0,0.0,0.0,0.0,1.0,0
1,1317,NaN,NaN,NaN,0,270,0,84,0,40,30,40,10,3.779645,1,0.0,0.0,0.0,1.0,2
2,1301,NaN,NaN,NaN,1,101,0,93,0,30,30,30,0,0.000000,0,0.0,0.0,0.0,1.0,0
3,1329,NaN,NaN,NaN,1,39,0,74,0,5,5,5,0,NaN,0,0.0,0.0,0.0,1.0,0
4,1370,NaN,NaN,NaN,1,6,0,85,0,0,0,0,0,NaN,0,0.0,1.0,0.0,1.0,1


In [7]:
external.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
H01_NUM,6339.0,1383.704685,767.419894,4.0,1184.0,1265.0,1386.000000,20295.000000
dbname,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
入家日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
結案日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
死亡標記,6339.0,0.280959,0.449503,0.0,0.0,0.0,1.000000,1.000000
觀察天數,6339.0,210.107588,109.964750,0.0,127.5,235.0,298.000000,365.000000
性別_is_male,6339.0,0.508440,0.499968,0.0,0.0,1.0,1.000000,1.000000
預估年齡,6339.0,78.607825,11.742108,1.0,72.0,81.0,87.000000,124.000000
DNR_flag,6339.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
ADL_總分_max,6339.0,29.695536,31.687105,0.0,0.0,20.0,50.000000,100.000000


In [8]:
ex_missing_info = external.isnull().sum().to_frame(name='Missing Count')
ex_missing_info['Missing Ratio'] = (ex_missing_info['Missing Count'] / len(external)).round(4)
ex_missing_info = ex_missing_info.sort_values(by='Missing Ratio', ascending=True)
ex_missing_info

,Missing Count,Missing Ratio
H01_NUM,0,0.0000
性別_is_male,0,0.0000
觀察天數,0,0.0000
死亡標記,0,0.0000
預估年齡,0,0.0000
ADL_first_score,0,0.0000
ADL_總分_max,0,0.0000
DNR_flag,0,0.0000
ADL_diff,0,0.0000
ADL_明顯惡化,0,0.0000


In [9]:
# read data and put it in a dataframe
# 在 google 工作表載入訓練資料 gsheets

# gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/14dXHX3xQlUWoe5uxm2DW-TRKoHee3AcHVVyBbljUMI0/edit?usp=sharing')
# worksheet = gsheets.worksheet("training_data_v1_no_matching")  # 指定分頁名稱

gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1t14jZVSsCyh993rZwtrtmzAeXeI4cPGzqGYNnt18jdw/edit?usp=sharing')
worksheet = gsheets.worksheet("training_data_v2")  # 指定分頁名稱

worksheet = worksheet.get_all_records()
df = pd.DataFrame(worksheet)
df = df.apply(lambda col: pd.to_numeric(col.astype(str).str.replace(',', '').str.strip(), errors='coerce'))
df.head()

,H01_NUM,dbname,入家日期,結案日期,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,ADL_first_score,ADL_last_score,ADL_diff,ADL_std,ADL_明顯惡化,意識清醒_max,使用呼吸輔具,活動假牙,使用管路,六個月內住院次數
0,1216,NaN,NaN,NaN,0,1056,1,81,0,0,0,0,0,NaN,0,0.0,0.0,0.0,1.0,NaN
1,1199,NaN,NaN,NaN,0,1278,1,61,0,0,0,0,0,0.0,0,0.0,0.0,0.0,1.0,NaN
2,1251,NaN,NaN,NaN,0,688,0,68,0,40,40,40,0,NaN,0,0.0,0.0,0.0,1.0,NaN
3,1205,NaN,NaN,NaN,0,1181,1,74,0,0,0,0,0,NaN,0,0.0,0.0,0.0,1.0,1.0
4,1206,NaN,NaN,NaN,0,1166,0,83,0,0,0,0,0,0.0,0,0.0,0.0,0.0,1.0,NaN


In [10]:
df.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
H01_NUM,27037.0,1500.801457,4160.182209,1.0,1159.0,1220.000000,1317.000000,100463.000000
dbname,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
入家日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
結案日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
死亡標記,27037.0,0.194992,0.396202,0.0,0.0,0.000000,0.000000,1.000000
觀察天數,27037.0,634.100529,437.553151,0.0,248.0,597.000000,1014.000000,1460.000000
性別_is_male,27037.0,0.485557,0.499801,0.0,0.0,0.000000,1.000000,1.000000
預估年齡,27037.0,79.388801,11.819412,0.0,72.0,82.000000,88.000000,125.000000
DNR_flag,27037.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
ADL_總分_max,27037.0,31.974886,33.277403,0.0,0.0,20.000000,55.000000,100.000000


In [11]:
df_missing_info = df.isnull().sum().to_frame(name='Missing Count')
df_missing_info['Missing Ratio'] = (df_missing_info['Missing Count'] / len(df)).round(4)
df_missing_info = df_missing_info.sort_values(by='Missing Ratio', ascending=True)
df_missing_info

,Missing Count,Missing Ratio
H01_NUM,0,0.0000
性別_is_male,0,0.0000
觀察天數,0,0.0000
死亡標記,0,0.0000
預估年齡,0,0.0000
ADL_first_score,0,0.0000
ADL_總分_max,0,0.0000
DNR_flag,0,0.0000
ADL_diff,0,0.0000
ADL_明顯惡化,0,0.0000


📊 如何證明死亡組與對照組的變項分布無顯著差異？

#標準化平均差 (Standardized Mean Difference, SMD) 或 t-test P-value 建議以 SMD 為主（不受樣本數影響）

In [12]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# 確保 '死亡標記' 是二元
df = df[df['死亡標記'].isin([0, 1])].copy()

# 使用 df_missing_info.index 作為變數清單
variables = df_missing_info.index

# 定義計算 SMD 與 P-value 的函數
def compute_stats(df, var, group_col='死亡標記'):
    group0 = df[df[group_col] == 0][var].dropna()
    group1 = df[df[group_col] == 1][var].dropna()

    mean0 = group0.mean()
    mean1 = group1.mean()
    var0 = group0.var(ddof=1)
    var1 = group1.var(ddof=1)

    std_pooled = np.sqrt((var0 + var1) / 2)
    smd = (mean1 - mean0) / std_pooled if std_pooled > 0 else np.nan

    # 計算 P-value
    try:
        stat, pval = ttest_ind(group1, group0, equal_var=False)
    except Exception:
        pval = np.nan

    # NaN 原因說明
    if std_pooled == 0 or np.isnan(smd):
        reason = "兩組皆為常數或無變異"
    elif len(group0.unique()) == 1 and len(group1.unique()) == 1 and group0.unique()[0] == group1.unique()[0]:
        reason = "兩組數值完全一致"
    else:
        reason = ""

    return mean1, mean0, smd, pval, reason

# 計算所有變數的 SMD + P-value
smd_pval_results = []
for var in variables:
    mean1, mean0, smd, pval, reason = compute_stats(df, var)
    smd_pval_results.append({
        "變數": var,
        "死亡組平均": round(mean1, 3) if not np.isnan(mean1) else "NaN",
        "對照組平均": round(mean0, 3) if not np.isnan(mean0) else "NaN",
        "SMD": round(smd, 3) if not np.isnan(smd) else "NaN",
        "P-value": round(pval, 4) if not np.isnan(pval) else "NaN",
        "說明": reason
    })

# 轉為 DataFrame
smd_df = pd.DataFrame(smd_pval_results)

# 分為兩個表：無 NaN 和 含 NaN
smd_df_no_nan = smd_df[smd_df["SMD"] != "NaN"].reset_index(drop=True)
smd_df_with_nan = smd_df[smd_df["SMD"] == "NaN"].reset_index(drop=True)

/usr/local/lib/python3.12/dist-packages/scipy/stats/_axis_nan_policy.py:579: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/usr/local/lib/python3.12/dist-packages/scipy/_lib/deprecation.py:234: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  return f(*args, **kwargs)


In [13]:
smd_df_no_nan

,變數,死亡組平均,對照組平均,SMD,P-value,說明
0,H01_NUM,1372.724,1531.825,-0.047,0.0,
1,性別_is_male,0.582,0.462,0.242,0.0,
2,觀察天數,64.42,772.091,-2.655,0.0,
3,預估年齡,80.159,79.202,0.082,0.0,
4,ADL_first_score,12.256,22.429,-0.395,0.0,
5,ADL_總分_max,15.436,35.981,-0.704,0.0,
6,ADL_diff,3.18,13.552,-0.62,0.0,
7,ADL_明顯惡化,0.099,0.41,-0.766,0.0,
8,ADL_last_score,15.436,35.981,-0.704,0.0,
9,使用呼吸輔具,0.306,0.33,-0.052,0.0009,


In [14]:
smd_df_with_nan

,變數,死亡組平均,對照組平均,SMD,P-value,說明
0,死亡標記,1.0,0.0,NaN,0.0,兩組皆為常數或無變異
1,DNR_flag,0.0,0.0,NaN,NaN,兩組皆為常數或無變異
2,意識清醒_max,0.0,0.0,NaN,NaN,兩組皆為常數或無變異
3,活動假牙,0.0,0.0,NaN,NaN,兩組皆為常數或無變異
4,使用管路,1.0,1.0,NaN,NaN,兩組皆為常數或無變異
5,dbname,NaN,NaN,NaN,NaN,兩組皆為常數或無變異
6,入家日期,NaN,NaN,NaN,NaN,兩組皆為常數或無變異
7,結案日期,NaN,NaN,NaN,NaN,兩組皆為常數或無變異


In [15]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, chi2_contingency

# 建立年齡區間
age_bins = [65, 70, 75, 80, 85, 90, np.inf]
age_labels = ['65-69', '70-74', '75-79', '80-84', '85-89', '90+']
df['年齡區間'] = pd.cut(df['預估年齡'], bins=age_bins, labels=age_labels, right=False)

long_table_rows = []

# 修正後的分類變項
categorical_vars = {
    '年齡區間': age_labels,
    '性別_is_male': [0, 1],  # 0=女性, 1=男性
    'DNR_flag': [1],
    '意識清醒_max': [1],
    '使用管路': [1],
    '活動假牙': [1],
    '使用呼吸輔具': [1]
}

continuous_vars = [
    'ADL_總分_max',
    'ADL_first_score',
    '六個月內住院次數'
]

# 處理類別變項
for var, subgroups in categorical_vars.items():
    for val in subgroups:
        sub_df = df[df[var] == val]
        sub_df = sub_df[sub_df['死亡標記'].isin([0, 1])]

        died = sub_df[sub_df['死亡標記'] == 1]
        survived = sub_df[sub_df['死亡標記'] == 0]

        died_n = len(died)
        surv_n = len(survived)
        total_n = died_n + surv_n

        died_pct = died_n / total_n * 100 if total_n > 0 else np.nan
        surv_pct = surv_n / total_n * 100 if total_n > 0 else np.nan

        # 顯示標籤
        if var == '性別_is_male':
            subgroup_label = '男性' if val == 1 else '女性'
            var_label = '性別'
        elif var == 'DNR_flag':
            subgroup_label = '是'
            var_label = 'DNR'
        elif var == '意識清醒_max':
            subgroup_label = '清醒'
            var_label = '意識程度'
        elif var == '使用管路':
            subgroup_label = '使用'
            var_label = '管路'
        elif var == '活動假牙':
            subgroup_label = '使用'
            var_label = '假牙'
        elif var == '使用呼吸輔具':
            subgroup_label = '使用'
            var_label = '呼吸輔具'
        elif var == '年齡區間':
            subgroup_label = val
            var_label = '年齡'
        else:
            subgroup_label = str(val)
            var_label = var

        # 計算 P-value（類別變項使用卡方）
        try:
            ct = pd.crosstab(df[var] == val, df['死亡標記'])
            if ct.shape == (2, 2):
                _, pval, _, _ = chi2_contingency(ct)
                pval = f"{pval:.3f}"
            else:
                pval = "NA"
        except:
            pval = "NA"

        long_table_rows.append({
            '變項': var_label,
            '分類': subgroup_label,
            '死亡組 n': died_n,
            '死亡組 %': f"{died_pct:.1f}%" if not np.isnan(died_pct) else 'NA',
            '存活組 n': surv_n,
            '存活組 %': f"{surv_pct:.1f}%" if not np.isnan(surv_pct) else 'NA',
            'P-value': pval
        })

# 處理連續變項
for var in continuous_vars:
    died = df[df['死亡標記'] == 1][var].dropna()
    survived = df[df['死亡標記'] == 0][var].dropna()

    died_n = len(died)
    surv_n = len(survived)

    died_mean, died_std = died.mean(), died.std()
    surv_mean, surv_std = survived.mean(), survived.std()

    # p-value
    if died_n > 1 and surv_n > 1:
        _, pval = ttest_ind(died, survived, equal_var=False)
        pval = f"{pval:.3f}"
    else:
        pval = "NA"

    long_table_rows.append({
        '變項': var,
        '分類': '',
        '死亡組 n': died_n,
        '死亡組 %': f"{died_mean:.1f} ± {died_std:.1f}",
        '存活組 n': surv_n,
        '存活組 %': f"{surv_mean:.1f} ± {surv_std:.1f}",
        'P-value': pval
    })

# 整理成表格
long_summary_df = pd.DataFrame(long_table_rows)

In [16]:
long_summary_df

,變項,分類,死亡組 n,死亡組 %,存活組 n,存活組 %,P-value
0,年齡,65-69,430,19.9%,1726,80.1%,0.606
1,年齡,70-74,549,17.5%,2594,82.5%,0.002
2,年齡,75-79,623,17.7%,2889,82.3%,0.005
3,年齡,80-84,1001,20.4%,3905,79.6%,0.081
4,年齡,85-89,1075,20.0%,4292,80.0%,0.282
5,年齡,90+,1073,21.1%,4021,78.9%,0.002
6,性別,女性,2203,15.8%,11706,84.2%,0.000
7,性別,男性,3069,23.4%,10059,76.6%,0.000
8,DNR,是,0,NA,0,NA,NA
9,意識程度,清醒,0,NA,0,NA,NA


In [17]:
from scipy.stats import ttest_ind
import pandas as pd
import numpy as np

# 為了進行統計檢定與格式整理，我們將針對每個年齡區間與性別群體，
# 分別產生「死亡組」與「非死亡組」的平均、標準差與 p-value。

# 定義要分析的連續變數（解決 NameError）
features = ['ADL_總分_max', 'ADL_first_score', '六個月內住院次數']

# 準備欄位
group_cols = ['年齡區間', '性別_is_male']  # 使用 '性別_is_male' 代替 '性別'
output_rows = []

# 每個變項進行比較（明確指定 observed=True 以解決 FutureWarning）
for (age_group, gender_code), group_df in df.groupby(group_cols, observed=True):
    # 映射性別碼為文字標籤
    gender_label = '男性' if gender_code == 1 else '女性'

    row = {'年齡區間': age_group, '性別': gender_label, '樣本數': len(group_df)}

    # 死亡/非死亡分開
    died = group_df[group_df['死亡標記'] == 1]
    survived = group_df[group_df['死亡標記'] == 0]

    row['死亡數'] = len(died)
    row['死亡率'] = len(died) / len(group_df) if len(group_df) > 0 else np.nan

    for feat in features:
        # 計算平均與標準差
        died_mean = died[feat].mean()
        died_std = died[feat].std()
        surv_mean = survived[feat].mean()
        surv_std = survived[feat].std()

        # t 檢定（排除 NaN）
        t_stat, p_val = ttest_ind(
            died[feat].dropna(), survived[feat].dropna(), equal_var=False
        ) if died[feat].dropna().size > 1 and survived[feat].dropna().size > 1 else (np.nan, np.nan)

        # 加入欄位（格式：mean ± std (p-value)）
        row[f'{feat}_死亡'] = f'{died_mean:.2f} ± {died_std:.2f}' if not np.isnan(died_mean) else 'NA'
        row[f'{feat}_存活'] = f'{surv_mean:.2f} ± {surv_std:.2f}' if not np.isnan(surv_mean) else 'NA'
        row[f'{feat}_p'] = f'{p_val:.3f}' if not np.isnan(p_val) else 'NA'

    output_rows.append(row)

# 整理成 DataFrame
formatted_df = pd.DataFrame(output_rows)

# 顯示結果
display(formatted_df)

,年齡區間,性別,樣本數,死亡數,死亡率,ADL_總分_max_死亡,ADL_總分_max_存活,ADL_總分_max_p,ADL_first_score_死亡,ADL_first_score_存活,ADL_first_score_p,六個月內住院次數_死亡,六個月內住院次數_存活,六個月內住院次數_p
0,65-69,女性,742,153,0.206199,19.08 ± 26.55,38.74 ± 34.32,0.000,16.27 ± 24.94,25.58 ± 31.29,0.000,1.98 ± 2.62,4.73 ± 9.91,0.000
1,65-69,男性,1414,277,0.195898,19.68 ± 27.15,38.42 ± 34.96,0.000,15.76 ± 23.73,24.48 ± 30.89,0.000,2.08 ± 1.65,4.86 ± 10.40,0.000
2,70-74,女性,1213,171,0.140973,16.35 ± 23.22,42.44 ± 36.72,0.000,12.78 ± 21.14,29.15 ± 33.95,0.000,1.87 ± 1.54,4.40 ± 7.98,0.000
3,70-74,男性,1930,378,0.195855,17.55 ± 23.62,40.66 ± 35.84,0.000,14.56 ± 21.70,25.56 ± 32.32,0.000,2.11 ± 2.11,4.73 ± 11.18,0.000
4,75-79,女性,1677,232,0.138342,16.08 ± 24.32,38.30 ± 35.15,0.000,13.51 ± 21.61,24.95 ± 31.35,0.000,2.20 ± 2.61,5.68 ± 16.86,0.000
5,75-79,男性,1835,391,0.213079,16.42 ± 24.29,38.19 ± 34.89,0.000,13.47 ± 21.61,23.84 ± 30.65,0.000,1.86 ± 1.49,6.15 ± 22.66,0.000
6,80-84,女性,2673,404,0.151141,15.45 ± 22.63,37.32 ± 34.75,0.000,11.78 ± 19.54,23.57 ± 30.97,0.000,2.25 ± 2.45,4.49 ± 10.60,0.000
7,80-84,男性,2233,597,0.267353,15.22 ± 22.74,35.94 ± 34.57,0.000,11.85 ± 19.93,22.37 ± 30.17,0.000,1.97 ± 2.06,4.27 ± 7.57,0.000
8,85-89,女性,3353,523,0.155980,14.08 ± 21.99,34.60 ± 33.01,0.000,10.33 ± 18.71,20.43 ± 27.94,0.000,1.92 ± 1.40,4.12 ± 6.73,0.000
9,85-89,男性,2014,552,0.274081,13.69 ± 22.59,33.66 ± 32.25,0.000,10.40 ± 19.15,19.16 ± 27.37,0.000,1.85 ± 1.40,4.77 ± 15.89,0.000


In [18]:
from scipy.stats import ttest_ind

# 為了進行統計檢定與格式整理，我們將針對每個年齡區間與性別群體，
# 分別產生「死亡組」與「非死亡組」的平均、標準差與 p-value。

# 準備欄位（修正：使用 '性別_is_male' 代替 '性別'）
group_cols = ['年齡區間', '性別_is_male']  # <- 這裡是關鍵修正
output_rows = []

# 每個變項進行比較
for (age_group, gender_code), group_df in df.groupby(group_cols):
    # 映射性別碼為文字標籤
    gender_label = '男性' if gender_code == 1 else '女性'

    row = {'年齡區間': age_group, '性別': gender_label, '樣本數': len(group_df)}

    # 死亡/非死亡分開
    died = group_df[group_df['死亡標記'] == 1]
    survived = group_df[group_df['死亡標記'] == 0]

    row['死亡數'] = len(died)
    row['死亡率'] = len(died) / len(group_df) if len(group_df) > 0 else np.nan

    for feat in features:  # 假設 features 是你要比較的欄位列表（如 ['ADL_總分_max', '六個月內住院次數'] 等）
        # 計算平均與標準差
        died_mean = died[feat].mean()
        died_std = died[feat].std()
        surv_mean = survived[feat].mean()
        surv_std = survived[feat].std()

        # t 檢定（排除 NaN）
        t_stat, p_val = ttest_ind(
            died[feat].dropna(), survived[feat].dropna(), equal_var=False
        ) if died[feat].dropna().size > 1 and survived[feat].dropna().size > 1 else (np.nan, np.nan)

        # 加入欄位（格式：mean ± std (p-value)）
        row[f'{feat}_死亡'] = f'{died_mean:.2f} ± {died_std:.2f}' if not np.isnan(died_mean) else 'NA'
        row[f'{feat}_存活'] = f'{surv_mean:.2f} ± {surv_std:.2f}' if not np.isnan(surv_mean) else 'NA'
        row[f'{feat}_p'] = f'{p_val:.3f}' if not np.isnan(p_val) else 'NA'

    output_rows.append(row)

# 整理成 DataFrame
formatted_df = pd.DataFrame(output_rows)

# 顯示結果（可選）
display(formatted_df)

/tmp/ipython-input-3029020933.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for (age_group, gender_code), group_df in df.groupby(group_cols):


,年齡區間,性別,樣本數,死亡數,死亡率,ADL_總分_max_死亡,ADL_總分_max_存活,ADL_總分_max_p,ADL_first_score_死亡,ADL_first_score_存活,ADL_first_score_p,六個月內住院次數_死亡,六個月內住院次數_存活,六個月內住院次數_p
0,65-69,女性,742,153,0.206199,19.08 ± 26.55,38.74 ± 34.32,0.000,16.27 ± 24.94,25.58 ± 31.29,0.000,1.98 ± 2.62,4.73 ± 9.91,0.000
1,65-69,男性,1414,277,0.195898,19.68 ± 27.15,38.42 ± 34.96,0.000,15.76 ± 23.73,24.48 ± 30.89,0.000,2.08 ± 1.65,4.86 ± 10.40,0.000
2,70-74,女性,1213,171,0.140973,16.35 ± 23.22,42.44 ± 36.72,0.000,12.78 ± 21.14,29.15 ± 33.95,0.000,1.87 ± 1.54,4.40 ± 7.98,0.000
3,70-74,男性,1930,378,0.195855,17.55 ± 23.62,40.66 ± 35.84,0.000,14.56 ± 21.70,25.56 ± 32.32,0.000,2.11 ± 2.11,4.73 ± 11.18,0.000
4,75-79,女性,1677,232,0.138342,16.08 ± 24.32,38.30 ± 35.15,0.000,13.51 ± 21.61,24.95 ± 31.35,0.000,2.20 ± 2.61,5.68 ± 16.86,0.000
5,75-79,男性,1835,391,0.213079,16.42 ± 24.29,38.19 ± 34.89,0.000,13.47 ± 21.61,23.84 ± 30.65,0.000,1.86 ± 1.49,6.15 ± 22.66,0.000
6,80-84,女性,2673,404,0.151141,15.45 ± 22.63,37.32 ± 34.75,0.000,11.78 ± 19.54,23.57 ± 30.97,0.000,2.25 ± 2.45,4.49 ± 10.60,0.000
7,80-84,男性,2233,597,0.267353,15.22 ± 22.74,35.94 ± 34.57,0.000,11.85 ± 19.93,22.37 ± 30.17,0.000,1.97 ± 2.06,4.27 ± 7.57,0.000
8,85-89,女性,3353,523,0.155980,14.08 ± 21.99,34.60 ± 33.01,0.000,10.33 ± 18.71,20.43 ± 27.94,0.000,1.92 ± 1.40,4.12 ± 6.73,0.000
9,85-89,男性,2014,552,0.274081,13.69 ± 22.59,33.66 ± 32.25,0.000,10.40 ± 19.15,19.16 ± 27.37,0.000,1.85 ± 1.40,4.77 ± 15.89,0.000


In [19]:
from scipy.stats import ttest_ind

# 為了進行統計檢定與格式整理，我們將針對每個年齡區間與性別群體，
# 分別產生「死亡組」與「非死亡組」的平均、標準差與 p-value。

# 準備欄位
group_cols = ['年齡區間', '性別_is_male']
output_rows = []

# 每個變項進行比較
for (age_group, gender), group_df in df.groupby(group_cols):
    row = {'年齡區間': age_group, '性別': gender, '樣本數': len(group_df)}

    # 死亡/非死亡分開
    died = group_df[group_df['死亡標記'] == 1]
    survived = group_df[group_df['死亡標記'] == 0]

    row['死亡數'] = len(died)
    row['死亡率'] = len(died) / len(group_df) if len(group_df) > 0 else np.nan

    for feat in features:
        # 計算平均與標準差
        died_mean = died[feat].mean()
        died_std = died[feat].std()
        surv_mean = survived[feat].mean()
        surv_std = survived[feat].std()

        # t 檢定（排除 NaN）
        t_stat, p_val = ttest_ind(
            died[feat].dropna(), survived[feat].dropna(), equal_var=False
        ) if died[feat].dropna().size > 1 and survived[feat].dropna().size > 1 else (np.nan, np.nan)

        # 加入欄位（格式：mean ± std (p-value)）
        row[f'{feat}_死亡'] = f'{died_mean:.2f} ± {died_std:.2f}' if not np.isnan(died_mean) else 'NA'
        row[f'{feat}_存活'] = f'{surv_mean:.2f} ± {surv_std:.2f}' if not np.isnan(surv_mean) else 'NA'
        row[f'{feat}_p'] = f'{p_val:.3f}' if not np.isnan(p_val) else 'NA'

    output_rows.append(row)

# 整理成 DataFrame
formatted_df = pd.DataFrame(output_rows)

/tmp/ipython-input-730470339.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for (age_group, gender), group_df in df.groupby(group_cols):


In [20]:
formatted_df

,年齡區間,性別,樣本數,死亡數,死亡率,ADL_總分_max_死亡,ADL_總分_max_存活,ADL_總分_max_p,ADL_first_score_死亡,ADL_first_score_存活,ADL_first_score_p,六個月內住院次數_死亡,六個月內住院次數_存活,六個月內住院次數_p
0,65-69,0,742,153,0.206199,19.08 ± 26.55,38.74 ± 34.32,0.000,16.27 ± 24.94,25.58 ± 31.29,0.000,1.98 ± 2.62,4.73 ± 9.91,0.000
1,65-69,1,1414,277,0.195898,19.68 ± 27.15,38.42 ± 34.96,0.000,15.76 ± 23.73,24.48 ± 30.89,0.000,2.08 ± 1.65,4.86 ± 10.40,0.000
2,70-74,0,1213,171,0.140973,16.35 ± 23.22,42.44 ± 36.72,0.000,12.78 ± 21.14,29.15 ± 33.95,0.000,1.87 ± 1.54,4.40 ± 7.98,0.000
3,70-74,1,1930,378,0.195855,17.55 ± 23.62,40.66 ± 35.84,0.000,14.56 ± 21.70,25.56 ± 32.32,0.000,2.11 ± 2.11,4.73 ± 11.18,0.000
4,75-79,0,1677,232,0.138342,16.08 ± 24.32,38.30 ± 35.15,0.000,13.51 ± 21.61,24.95 ± 31.35,0.000,2.20 ± 2.61,5.68 ± 16.86,0.000
5,75-79,1,1835,391,0.213079,16.42 ± 24.29,38.19 ± 34.89,0.000,13.47 ± 21.61,23.84 ± 30.65,0.000,1.86 ± 1.49,6.15 ± 22.66,0.000
6,80-84,0,2673,404,0.151141,15.45 ± 22.63,37.32 ± 34.75,0.000,11.78 ± 19.54,23.57 ± 30.97,0.000,2.25 ± 2.45,4.49 ± 10.60,0.000
7,80-84,1,2233,597,0.267353,15.22 ± 22.74,35.94 ± 34.57,0.000,11.85 ± 19.93,22.37 ± 30.17,0.000,1.97 ± 2.06,4.27 ± 7.57,0.000
8,85-89,0,3353,523,0.155980,14.08 ± 21.99,34.60 ± 33.01,0.000,10.33 ± 18.71,20.43 ± 27.94,0.000,1.92 ± 1.40,4.12 ± 6.73,0.000
9,85-89,1,2014,552,0.274081,13.69 ± 22.59,33.66 ± 32.25,0.000,10.40 ± 19.15,19.16 ± 27.37,0.000,1.85 ± 1.40,4.77 ± 15.89,0.000


In [21]:
long_summary_df

,變項,分類,死亡組 n,死亡組 %,存活組 n,存活組 %,P-value
0,年齡,65-69,430,19.9%,1726,80.1%,0.606
1,年齡,70-74,549,17.5%,2594,82.5%,0.002
2,年齡,75-79,623,17.7%,2889,82.3%,0.005
3,年齡,80-84,1001,20.4%,3905,79.6%,0.081
4,年齡,85-89,1075,20.0%,4292,80.0%,0.282
5,年齡,90+,1073,21.1%,4021,78.9%,0.002
6,性別,女性,2203,15.8%,11706,84.2%,0.000
7,性別,男性,3069,23.4%,10059,76.6%,0.000
8,DNR,是,0,NA,0,NA,NA
9,意識程度,清醒,0,NA,0,NA,NA
